In [1]:
import requests
import entities.entities as en
from src.config import ORION_URL, NGSI_LD_ENDPOINT, headers

In [2]:
# Check Orion Context Broker version
def test_connection():
    """Check if Orion Context Broker is accessible"""
    try:
        # Send GET request to version endpoint
        response = requests.get(f"{ORION_URL}/version")
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            version_info = response.json()
            print(f"Orion Version: {version_info.get('orion version', 'Unknown')}")
            print(f"Orion-LD Version: {version_info.get('orionld version', 'Unknown')}")
        else:
            print("Failed to connect to Orion")
            
    except Exception as e:
        print(f"Connection error: {e}")

# Run the test
test_connection()

Status Code: 200
Orion Version: 1.15.0-next
Orion-LD Version: 1.4.0


In [3]:
# def function for entity query test

def get_entity_types():
    """Get all available entity types"""
    try:
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/types", headers=headers)
        print(f"Status Code: {response.status_code}")
        if response.status_code == 200:
            types_data = response.json()
                         
            return types_data
        else:
            print(f"Failed: {response.text}")
            return []
    except Exception as e:
        print(f"Error: {e}")
        return []

def get_all_entities():
    """Get all entities from Orion Context Broker"""
    try:
        # Add local=true to avoid "too broad query" error
        params = {
            "limit": 1000,
            "local": "true"
        }
        
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities", 
                              headers=headers, 
                              params=params)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Retrieved {len(entities)} entities")
            
            # Display basic info about each entity
            for i, entity in enumerate(entities):
                print(f"\nEntity {i+1}:")
                print(f"  ID: {entity.get('id')}")
                
                # Handle type field in NGSI-LD format
                entity_type = entity.get('type')
                if isinstance(entity_type, list):
                    entity_type = entity_type[0] if entity_type else 'Unknown'
                print(f"  Type: {entity_type}")
                
                # Count attributes (exclude @context, id, type)
                excluded_keys = ['@context', 'id', 'type']
                attr_count = len([k for k in entity.keys() if k not in excluded_keys])
                print(f"  Attributes: {attr_count}")
                
                # Show @context if present
                if '@context' in entity:
                    print(f"  Context: {entity.get('@context')}")
            
            return entities
        else:
            print("Failed to get entities")
            print(f"Response: {response.text}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []
    
def query_entity_by_id(entity_id):
    """Query entity by exact ID"""
    try:
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities/{entity_id}", 
                              headers=headers)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            entity = response.json()          
            return entity
        else:
            print(f"Entity not found: {response.text}")
            return None
            
    except Exception as e:
        print(f"Error: {e}")
        return None

def query_entities_by_type(entity_type):
    """Query entities by type"""
    try:
        params = {
            "type": entity_type,
            "limit": 10     # limit 10 outputs
        }
        
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities", 
                              headers=headers, 
                              params=params)
        print(f"Status Code: {response.status_code}")
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Found {len(entities)} entities of type '{entity_type}'")
            
            for i, entity in enumerate(entities):
                print(f"\nEntity {i+1}:")
                print(f"  ID: {entity.get('id')}")
                print(f"  Type: {entity.get('type')}")
            
            return entities
        else:
            print(f"Query failed: {response.text}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []

def query_entities_by_attribute(attribute_name, attribute_value):
    """Query entities by attribute value"""
    try:
        params = {
            "q": f'{attribute_name}=="{attribute_value}"',
            "limit": 10
        }
        
        response = requests.get(f"{ORION_URL}/ngsi-ld/v1/entities", 
                              headers=headers, 
                              params=params)
        print(f"Status Code: {response.status_code}")
        print(f"Query: {params['q']}")
        
        if response.status_code == 200:
            entities = response.json()
            print(f"Found {len(entities)} entities with {attribute_name}={attribute_value}")
            
            for i, entity in enumerate(entities):
                print(f"\nEntity {i+1}:")
                print(f"  ID: {entity.get('id')}")
                print(f"  Type: {entity.get('type')}")
                if attribute_name in entity:
                    attr_value = entity[attribute_name]
                    if isinstance(attr_value, dict) and 'value' in attr_value:
                        print(f"  {attribute_name}: {attr_value['value']}")
                    else:
                        print(f"  {attribute_name}: {attr_value}")
            
            return entities
        else:
            print(f"Query failed: {response.text}")
            return []
            
    except Exception as e:
        print(f"Error: {e}")
        return []



In [4]:
get_entity_types()

Status Code: 200


{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
 'id': 'urn:ngsi-ld:EntityTypeList:d5552c1a-664a-11f0-874e-ee53fd41f800',
 'type': 'EntityTypeList',
 'typeList': ['ChargingPoint',
  'ChargingPointStatus',
  'ChargingSession',
  'ChargingStation',
  'ChargingStationStatus',
  'E-Vehicle']}

In [5]:
get_all_entities()

Status Code: 200
Retrieved 1000 entities

Entity 1:
  ID: urn:ngsi-ld:ChargingPoint:E4C-001-CP-001
  Type: ChargingPoint
  Attributes: 8
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 2:
  ID: urn:ngsi-ld:E-Vehicle:EV-001
  Type: E-Vehicle
  Attributes: 8
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 3:
  ID: urn:ngsi-ld:ChargingSession:session-512840826
  Type: ChargingSession
  Attributes: 14
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 4:
  ID: urn:ngsi-ld:ChargingStation:ParisSaclay30-1
  Type: ChargingStation
  Attributes: 7
  Context: https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld

Entity 5:
  ID: urn:ngsi-ld:ChargingStation:Building103
  Type: ChargingStation
  Attributes: 4
  Context

[{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingPoint:E4C-001-CP-001',
  'type': 'ChargingPoint',
  'chargingType': {'type': 'Property', 'value': 'AC-Level2'},
  'communicationProtocol': {'type': 'Property', 'value': 'OCPP1.6J'},
  'connectorTypes': {'type': 'Property', 'value': 'Type2'},
  'dateCreated': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-07-21T17:30:59.846070Z'}},
  'pointId': {'type': 'Property', 'value': 'CP-030-1-001'},
  'powerCapabilities': {'type': 'Property',
   'value': {'maxACPower': 22,
    'maxDCPower': 0,
    'maxVoltage': 400,
    'maxCurrent': 32,
    'phases': 3}},
  'refChargingPointStatus': {'type': 'Relationship',
   'object': 'urn:ngsi-ld:ChargingPointStatus:E4C-001-CP-001-status-0001'},
  'refChargingStation': {'type': 'Relationship',
   'object': 'urn:ngsi-ld:ChargingStation:E4C-001'}},
 {'@context': 'https://raw.githubuserc

In [6]:
query_entity_by_id("urn:ngsi-ld:ChargingSession:9105d559-f024-45f6-a3e7-ac70bca29978")

Status Code: 200


{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
 'id': 'urn:ngsi-ld:ChargingSession:9105d559-f024-45f6-a3e7-ac70bca29978',
 'type': 'ChargingSession',
 'refChargingPoint': {'type': 'Relationship',
  'object': 'urn:ngsi-ld:ChargingPoint:Building103-P-3'},
 'sessionId': {'type': 'Property',
  'value': '9105d559-f024-45f6-a3e7-ac70bca29978'},
 'sessionStatus': {'type': 'Property', 'value': 'ended'},
 'sessionType': {'type': 'Property', 'value': 'AC-Level1'},
 'sessionStartTime': {'type': 'Property',
  'value': {'@type': 'DateTime', '@value': '2025-05-20T15:30:20+02:00'},
  'observedAt': '2025-05-20T13:30:20.000Z'},
 'sessionEndTime': {'type': 'Property',
  'value': {'@type': 'DateTime', '@value': '2025-05-20T17:30:57+02:00'},
  'observedAt': '2025-05-20T15:30:57.000Z'},
 'sessionDuration': {'type': 'Property',
  'value': 121,
  'units': {'type': 'Property', 'value': 'MIN'},
  'observedAt': '2025-05-20T15:30:57.000Z

In [7]:
query_entities_by_type("ChargingStation")


Status Code: 200
Found 2 entities of type 'ChargingStation'

Entity 1:
  ID: urn:ngsi-ld:ChargingStation:ParisSaclay30-1
  Type: ChargingStation

Entity 2:
  ID: urn:ngsi-ld:ChargingStation:Building103
  Type: ChargingStation


[{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingStation:ParisSaclay30-1',
  'type': 'ChargingStation',
  'dateCreated': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-07-21T17:45:14.850459Z'}},
  'maxPowerCapacity': {'type': 'Property',
   'value': 22,
   'units': {'type': 'Property', 'value': 'kW'}},
  'ratedVoltage': {'type': 'Property',
   'value': 400,
   'units': {'type': 'Property', 'value': 'V'}},
  'socketType': {'type': 'Property', 'value': 'Type2'},
  'stationName': {'type': 'Property', 'value': 'IPP - Drahi - X'},
  'totalChargingPoints': {'type': 'Property', 'value': 1},
  'location': {'type': 'GeoProperty',
   'value': {'type': 'Point', 'coordinates': [2.212206, 48.712779]}}},
 {'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingStation:Building103

In [8]:
query_entities_by_attribute("sessionType", "AC-Level1")

Status Code: 200
Query: sessionType=="AC-Level1"
Found 10 entities with sessionType=AC-Level1

Entity 1:
  ID: urn:ngsi-ld:ChargingSession:9105d559-f024-45f6-a3e7-ac70bca29978
  Type: ChargingSession
  sessionType: AC-Level1

Entity 2:
  ID: urn:ngsi-ld:ChargingSession:21728257-9c03-4152-98f4-4bb7257ad8cc
  Type: ChargingSession
  sessionType: AC-Level1

Entity 3:
  ID: urn:ngsi-ld:ChargingSession:c048a583-10a9-4ac2-8252-3a1fd1220570
  Type: ChargingSession
  sessionType: AC-Level1

Entity 4:
  ID: urn:ngsi-ld:ChargingSession:a5886ef5-c3c8-4f35-a9fd-0c841ae1d912
  Type: ChargingSession
  sessionType: AC-Level1

Entity 5:
  ID: urn:ngsi-ld:ChargingSession:ac21108c-142b-4cd8-b255-7bb11cb941e0
  Type: ChargingSession
  sessionType: AC-Level1

Entity 6:
  ID: urn:ngsi-ld:ChargingSession:54899683-e671-477b-b09b-c0c4429eab3f
  Type: ChargingSession
  sessionType: AC-Level1

Entity 7:
  ID: urn:ngsi-ld:ChargingSession:4946b96d-5927-4ebb-a062-411903daa338
  Type: ChargingSession
  sessionType:

[{'@context': 'https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld',
  'id': 'urn:ngsi-ld:ChargingSession:9105d559-f024-45f6-a3e7-ac70bca29978',
  'type': 'ChargingSession',
  'authenticationId': {'type': 'Property', 'value': 'polytechnique8'},
  'chargingDuration': {'type': 'Property',
   'value': 109,
   'units': {'type': 'Property', 'value': 'MIN'},
   'observedAt': '2025-05-20T15:19:17.000Z'},
  'chargingEndTime': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-05-20T17:19:17+02:00'},
   'observedAt': '2025-05-20T15:19:17.000Z'},
  'chargingStartTime': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-05-20T15:30:22+02:00'},
   'observedAt': '2025-05-20T13:30:22.000Z'},
  'dateCreated': {'type': 'Property',
   'value': {'@type': 'DateTime', '@value': '2025-05-20T15:30:20+02:00'}},
  'energyConsumed': {'type': 'Property',
   'value': 1.391,
   'units': {'type': 'Property', 'value': 'kW


Headers:
Content-Type: application/json
Accept: application/ld+json
Link: <https://raw.githubusercontent.com/chzh63315/DigiEV/refs/heads/main/contexts/datamodels.context-ngsi.jsonld>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"


query session duration > 500

Get

http://localhost:1026/ngsi-ld/v1/entities?type=ChargingSession&q=sessionDuration>500&limit=20



http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=after&timeAt=2024-12-31T23:59:59Z&timeproperty=observedAt&attrs=sessionStartTime&limit=20

http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=after&timeAt=2024-12-31T23:59:59Z&timeproperty=observedAt&attrs=energyConsumed&limit=20

http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=after&timeAt=2024-12-31T23:59:59Z&timeproperty=observedAt&attrs=energyConsumed,chargingDuration&limit=20

http://localhost:8080/temporal/entities/?type=ChargingSession&timerel=between&timeAt=2025-03-01T00:00:00Z&endTimeAt=2025-05-31T23:59:59Z&timeproperty=observedAt&attrs=sessionEndTime&limit=100



